In [1]:
import math_dataset 
from math_dataset import MathDatasetManager
import torch
import torch.optim as optim
from torch.utils import data
from math_dataset import (
    question_answer_to_position_batch_collate_fn
)
import model_process


import utils

%matplotlib notebook  

print("Torch Version", torch.__version__)

%load_ext autoreload
%autoreload 2

Torch Version 1.5.0+cu101


## Math Dataset Manager

Esta es una clase que nos permite cargar de manera sencilla cualquiera de los problemas disponilbles en el dataset de  problemas matematicos mathematics_dataset-v1.0 Disponible en este enlace https://console.cloud.google.com/storage/browser/_details/mathematics-dataset/mathematics_dataset-v1.0.tar.gz 

#### Es requerido descargar todo el dataset para poder realizar la ejecucion.

In [2]:
#Indicamos donde se encuentra el data set
mdsmgr = MathDatasetManager(
  "C:\\Users\\Jesús\\Documents\\PC5\\TorchDemo\\mathematics_dataset-v1.0\\mathematics_dataset-v1.0\\"
)

initialized MultiFilesMathDataset with categories ['algebra', 'arithmetic', 'calculus', 'comparison', 'measurement', 'numbers', 'polynomials', 'probability'] and types ['train-easy', 'train-medium', 'train-hard', 'interpolate', 'extrapolate']


### Los tipos de archivos de datos se clasifican por dificultad y grado de generalización.

In [3]:

print("types", list(mdsmgr.get_types()))

types ['train-easy', 'train-medium', 'train-hard', 'interpolate', 'extrapolate']


### Existen problemas en multiples areas

In [4]:
print("categories", list(mdsmgr.get_categories()))

categories ['algebra', 'arithmetic', 'calculus', 'comparison', 'measurement', 'numbers', 'polynomials', 'probability']


### Inicilaizacion de pytorch

In [5]:
seed = 1
torch.manual_seed(seed)
device = torch.device("cuda")
print("device", device)

device cuda


## Entrenando  add_or_sub nivel "Hard"

### Create an experiment with a name and a unique ID

In [6]:
exp_name = "add_or_sub" # "math_ds_algebra_linear_1d_easy"
unique_id = "2021-07-25" # "2019-05-25_0900"

### Construimos el dataset para entrenamiento

#### Usaremos 'add_or_sub', 'train-hard'

In [7]:
ds = mdsmgr.build_dataset_from_module(
    'arithmetic', 'add_or_sub', 'train-hard'
)
print("train-easy dataset size", len(ds))

train-easy dataset size 666666


C:\Users\Jesús\Documents\PC2\TorchDemo\hs-math-nlp\torch\lib\site-packages\pandas\core\frame.py:5042: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


#### El dataset de interpolacion nos servira como validacion.

In [8]:
ds_interpolate = mdsmgr.build_dataset_from_module(
    'arithmetic', 'add_or_sub', 'interpolate'
)
print("interpolate dataset size", len(ds_interpolate))

interpolate dataset size 10000


### Creamos un modelo transformer 

La arquitectura de este modelo esta descrita con detalle en la seccion 
Here we test the best model found in the paper: a multi-head self-attention transformer to give a default sample.


In [9]:
model = utils.build_transformer()

In [10]:
optimizer = optim.Adam(model.parameters(), lr=6e-6, betas=(0.9, 0.995), eps=1e-9)

### Dataloaders con multiples workers

In [11]:
# Se divide los datos en 90% entrenamiento y 10% validacion
train_ds, val_ds = math_dataset.random_split_dataset(ds, split_rate=0.9)

train_loader = data.DataLoader(
    train_ds, batch_size=128, shuffle=True, num_workers=12,
    collate_fn=question_answer_to_position_batch_collate_fn)

val_loader = data.DataLoader(
    val_ds, batch_size=128, shuffle=False, num_workers=12,
    collate_fn=question_answer_to_position_batch_collate_fn)

interpolate_loader = data.DataLoader(
    ds_interpolate, batch_size=128, shuffle=False, num_workers=12,
    collate_fn=question_answer_to_position_batch_collate_fn)


In [15]:
# Celda para cargar un modelo a partir de un checkpoint

#import checkpoints

# build default transformer model
#model = utils.build_transformer()

#model_exp_name = "linear_algebra" # "math_ds_algebra_linear_1d_easy"
#model_unique_id  = "2020-07-22" # "2019-05-25_0900"
#model_exp_name = 'math_ds_algebra_linear_1d_easy'
#model_unique_id = '2019-10-27_2300'
# restore best validation model from checkpoint
#_ = checkpoints.restore_checkpoint(".\\checkpoints\\checkpoint_b37504_e7.pth","", model=model)


### Ejecutamos el entrenamiento

In [13]:
model = model.to(device)


In [14]:
model_process.train(
    name = exp_name +"-" + unique_id,
    model = model,
    training_data= train_loader,
    validation_data = val_loader,
    interpolate_data=interpolate_loader,
    optimizer = optimizer,
    device = device,
    epochs=8,
    tb=None,
    log_interval=100)

~~~ Beginning Training ~~~~
Start epoch: 0, Start batch: 0, Max batch: None
[ Epoch: 0 / 8, Run Batch: 0 / None]
Batch: 0. Acc: 0.011730. Loss: 5.227549. Batch_acc: 0.011730. Batch_loss: 5.227549 
Batch: 1. Acc: 0.019573. Loss: 5.078589. Batch_acc: 0.027357. Batch_loss: 4.930756 
Batch: 2. Acc: 0.036892. Loss: 4.888388. Batch_acc: 0.071765. Batch_loss: 4.505412 
Batch: 3. Acc: 0.048684. Loss: 4.726118. Batch_acc: 0.083867. Batch_loss: 4.241954 
Batch: 4. Acc: 0.055477. Loss: 4.584731. Batch_acc: 0.083037. Batch_loss: 4.011135 
Batch: 5. Acc: 0.061344. Loss: 4.453219. Batch_acc: 0.090855. Batch_loss: 3.791699 
Batch: 6. Acc: 0.064137. Loss: 4.332433. Batch_acc: 0.080271. Batch_loss: 3.634551 
Batch: 7. Acc: 0.066735. Loss: 4.224350. Batch_acc: 0.084677. Batch_loss: 3.477855 
Batch: 8. Acc: 0.069298. Loss: 4.123463. Batch_acc: 0.089647. Batch_loss: 3.322553 
Batch: 9. Acc: 0.072976. Loss: 4.033534. Batch_acc: 0.106930. Batch_loss: 3.203272 
Batch: 10. Acc: 0.075828. Loss: 3.952317. Batch

Batch: 96. Acc: 0.130339. Loss: 2.711562. Batch_acc: 0.152632. Batch_loss: 2.449801 
Batch: 97. Acc: 0.130475. Loss: 2.708751. Batch_acc: 0.143503. Batch_loss: 2.441198 
Batch: 98. Acc: 0.130731. Loss: 2.706103. Batch_acc: 0.155280. Batch_loss: 2.451573 
Batch: 99. Acc: 0.130877. Loss: 2.703791. Batch_acc: 0.145821. Batch_loss: 2.468104 
Batch: 100. Acc: 0.131144. Loss: 2.701082. Batch_acc: 0.156983. Batch_loss: 2.438200 
Batch: 101. Acc: 0.131427. Loss: 2.698730. Batch_acc: 0.160279. Batch_loss: 2.459063 
Batch: 102. Acc: 0.131773. Loss: 2.696201. Batch_acc: 0.167245. Batch_loss: 2.436856 
Batch: 103. Acc: 0.132127. Loss: 2.693534. Batch_acc: 0.168682. Batch_loss: 2.418709 
Batch: 104. Acc: 0.132377. Loss: 2.690947. Batch_acc: 0.158537. Batch_loss: 2.419548 
Batch: 105. Acc: 0.132619. Loss: 2.688648. Batch_acc: 0.158107. Batch_loss: 2.446695 
Batch: 106. Acc: 0.132711. Loss: 2.686417. Batch_acc: 0.142525. Batch_loss: 2.447489 
Batch: 107. Acc: 0.132905. Loss: 2.684240. Batch_acc: 0.15

Batch: 192. Acc: 0.165023. Loss: 2.518801. Batch_acc: 0.233100. Batch_loss: 2.240252 
Batch: 193. Acc: 0.165337. Loss: 2.517332. Batch_acc: 0.224341. Batch_loss: 2.241188 
Batch: 194. Acc: 0.165625. Loss: 2.515889. Batch_acc: 0.220957. Batch_loss: 2.238852 
Batch: 195. Acc: 0.165971. Loss: 2.514502. Batch_acc: 0.234880. Batch_loss: 2.238497 
Batch: 196. Acc: 0.166341. Loss: 2.513063. Batch_acc: 0.239420. Batch_loss: 2.229047 
Batch: 197. Acc: 0.166680. Loss: 2.511684. Batch_acc: 0.233661. Batch_loss: 2.238823 
Batch: 198. Acc: 0.166969. Loss: 2.510322. Batch_acc: 0.224267. Batch_loss: 2.240872 
Batch: 199. Acc: 0.167245. Loss: 2.509036. Batch_acc: 0.223335. Batch_loss: 2.246944 
Batch: 200. Acc: 0.167565. Loss: 2.507583. Batch_acc: 0.231926. Batch_loss: 2.215740 
Batch: 201. Acc: 0.167877. Loss: 2.506270. Batch_acc: 0.230903. Batch_loss: 2.240991 
Batch: 202. Acc: 0.168119. Loss: 2.505044. Batch_acc: 0.217926. Batch_loss: 2.253050 
Batch: 203. Acc: 0.168430. Loss: 2.503738. Batch_acc: 

Batch: 280. Acc: 0.186494. Loss: 2.424831. Batch_acc: 0.243151. Batch_loss: 2.192437 
Batch: 281. Acc: 0.186665. Loss: 2.424046. Batch_acc: 0.234152. Batch_loss: 2.205135 
Batch: 282. Acc: 0.186832. Loss: 2.423288. Batch_acc: 0.233352. Batch_loss: 2.212170 
Batch: 283. Acc: 0.187016. Loss: 2.422621. Batch_acc: 0.239080. Batch_loss: 2.234367 
Batch: 284. Acc: 0.187215. Loss: 2.421842. Batch_acc: 0.243028. Batch_loss: 2.202984 
Batch: 285. Acc: 0.187401. Loss: 2.421088. Batch_acc: 0.241481. Batch_loss: 2.202091 
Batch: 286. Acc: 0.187581. Loss: 2.420349. Batch_acc: 0.239650. Batch_loss: 2.206257 
Batch: 287. Acc: 0.187758. Loss: 2.419659. Batch_acc: 0.238754. Batch_loss: 2.221364 
Batch: 288. Acc: 0.187961. Loss: 2.418887. Batch_acc: 0.245594. Batch_loss: 2.199306 
Batch: 289. Acc: 0.188189. Loss: 2.418107. Batch_acc: 0.253273. Batch_loss: 2.195477 
Batch: 290. Acc: 0.188399. Loss: 2.417289. Batch_acc: 0.247486. Batch_loss: 2.187320 
Batch: 291. Acc: 0.188566. Loss: 2.416506. Batch_acc: 

Batch: 376. Acc: 0.200684. Loss: 2.364721. Batch_acc: 0.247405. Batch_loss: 2.176139 
Batch: 377. Acc: 0.200823. Loss: 2.364215. Batch_acc: 0.253303. Batch_loss: 2.174044 
Batch: 378. Acc: 0.200938. Loss: 2.363721. Batch_acc: 0.243721. Batch_loss: 2.178491 
Batch: 379. Acc: 0.201015. Loss: 2.363249. Batch_acc: 0.230199. Batch_loss: 2.186127 
Batch: 380. Acc: 0.201114. Loss: 2.362784. Batch_acc: 0.238979. Batch_loss: 2.184648 
Batch: 381. Acc: 0.201219. Loss: 2.362330. Batch_acc: 0.240847. Batch_loss: 2.190408 
Batch: 382. Acc: 0.201287. Loss: 2.361905. Batch_acc: 0.227804. Batch_loss: 2.196945 
Batch: 383. Acc: 0.201424. Loss: 2.361430. Batch_acc: 0.253586. Batch_loss: 2.180168 
Batch: 384. Acc: 0.201530. Loss: 2.360897. Batch_acc: 0.242147. Batch_loss: 2.157825 
Batch: 385. Acc: 0.201651. Loss: 2.360455. Batch_acc: 0.247991. Batch_loss: 2.190718 
Batch: 386. Acc: 0.201767. Loss: 2.359987. Batch_acc: 0.246136. Batch_loss: 2.180077 
Batch: 387. Acc: 0.201871. Loss: 2.359529. Batch_acc: 

...

Batch: 1216. Acc: 0.563308. Loss: 1.201652. Batch_acc: 0.570281. Batch_loss: 1.200721 
Batch: 1217. Acc: 0.563299. Loss: 1.201671. Batch_acc: 0.553155. Batch_loss: 1.223748 
Batch: 1218. Acc: 0.563329. Loss: 1.201576. Batch_acc: 0.598412. Batch_loss: 1.087555 
Batch: 1219. Acc: 0.563344. Loss: 1.201538. Batch_acc: 0.582293. Batch_loss: 1.156237 
Batch: 1220. Acc: 0.563346. Loss: 1.201500. Batch_acc: 0.565545. Batch_loss: 1.154702 
Batch: 1221. Acc: 0.563340. Loss: 1.201523. Batch_acc: 0.555492. Batch_loss: 1.229556 
Batch: 1222. Acc: 0.563346. Loss: 1.201512. Batch_acc: 0.570845. Batch_loss: 1.187849 
Batch: 1223. Acc: 0.563360. Loss: 1.201472. Batch_acc: 0.580022. Batch_loss: 1.154562 
Batch: 1224. Acc: 0.563371. Loss: 1.201435. Batch_acc: 0.576744. Batch_loss: 1.155582 
Batch: 1225. Acc: 0.563355. Loss: 1.201458. Batch_acc: 0.544084. Batch_loss: 1.230125 
Batch: 1226. Acc: 0.563349. Loss: 1.201460. Batch_acc: 0.555426. Batch_loss: 1.203972 
Batch: 1227. Acc: 0.563356. Loss: 1.201433.

Batch: 1304. Acc: 0.563432. Loss: 1.201096. Batch_acc: 0.583007. Batch_loss: 1.121602 
Batch: 1305. Acc: 0.563447. Loss: 1.201031. Batch_acc: 0.583001. Batch_loss: 1.117312 
Batch: 1306. Acc: 0.563448. Loss: 1.201026. Batch_acc: 0.564311. Batch_loss: 1.194467 
Batch: 1307. Acc: 0.563439. Loss: 1.201054. Batch_acc: 0.552179. Batch_loss: 1.236631 
Batch: 1308. Acc: 0.563444. Loss: 1.201052. Batch_acc: 0.569014. Batch_loss: 1.199242 
Batch: 1309. Acc: 0.563451. Loss: 1.201031. Batch_acc: 0.572493. Batch_loss: 1.173247 
Batch: 1310. Acc: 0.563475. Loss: 1.200949. Batch_acc: 0.594624. Batch_loss: 1.098561 
Batch: 1311. Acc: 0.563470. Loss: 1.200984. Batch_acc: 0.555944. Batch_loss: 1.247347 
Batch: 1312. Acc: 0.563472. Loss: 1.200962. Batch_acc: 0.566015. Batch_loss: 1.171964 
Batch: 1313. Acc: 0.563466. Loss: 1.200990. Batch_acc: 0.556078. Batch_loss: 1.238022 
Batch: 1314. Acc: 0.563458. Loss: 1.201021. Batch_acc: 0.552431. Batch_loss: 1.242382 
Batch: 1315. Acc: 0.563449. Loss: 1.201053.

Batch: 1399. Acc: 0.563707. Loss: 1.200108. Batch_acc: 0.556391. Batch_loss: 1.223362 
Batch: 1400. Acc: 0.563714. Loss: 1.200100. Batch_acc: 0.573844. Batch_loss: 1.189043 
Batch: 1401. Acc: 0.563715. Loss: 1.200093. Batch_acc: 0.565217. Batch_loss: 1.189826 
Batch: 1402. Acc: 0.563728. Loss: 1.200055. Batch_acc: 0.581639. Batch_loss: 1.146948 
Batch: 1403. Acc: 0.563733. Loss: 1.200024. Batch_acc: 0.571591. Batch_loss: 1.156373 
Batch: 1404. Acc: 0.563730. Loss: 1.200042. Batch_acc: 0.558980. Batch_loss: 1.226849 
Batch: 1405. Acc: 0.563715. Loss: 1.200081. Batch_acc: 0.543054. Batch_loss: 1.254149 
Batch: 1406. Acc: 0.563729. Loss: 1.200035. Batch_acc: 0.582673. Batch_loss: 1.137018 
Batch: 1407. Acc: 0.563729. Loss: 1.200024. Batch_acc: 0.563793. Batch_loss: 1.184266 
Batch: 1408. Acc: 0.563739. Loss: 1.199988. Batch_acc: 0.577778. Batch_loss: 1.149889 
Batch: 1409. Acc: 0.563729. Loss: 1.200007. Batch_acc: 0.549971. Batch_loss: 1.225489 
Batch: 1410. Acc: 0.563719. Loss: 1.200026.

Batch: 1494. Acc: 0.563969. Loss: 1.199354. Batch_acc: 0.582115. Batch_loss: 1.141438 
Batch: 1495. Acc: 0.563968. Loss: 1.199342. Batch_acc: 0.561611. Batch_loss: 1.181935 
Batch: 1496. Acc: 0.563966. Loss: 1.199349. Batch_acc: 0.562315. Batch_loss: 1.208739 
Batch: 1497. Acc: 0.563977. Loss: 1.199318. Batch_acc: 0.579279. Batch_loss: 1.153160 
Batch: 1498. Acc: 0.563965. Loss: 1.199352. Batch_acc: 0.546416. Batch_loss: 1.251270 
Batch: 1499. Acc: 0.563985. Loss: 1.199298. Batch_acc: 0.593096. Batch_loss: 1.121101 
Batch: 1500. Acc: 0.563983. Loss: 1.199312. Batch_acc: 0.560838. Batch_loss: 1.219624 
Batch: 1501. Acc: 0.563989. Loss: 1.199306. Batch_acc: 0.573079. Batch_loss: 1.190048 
Batch: 1502. Acc: 0.563992. Loss: 1.199286. Batch_acc: 0.568493. Batch_loss: 1.168851 
Batch: 1503. Acc: 0.563983. Loss: 1.199308. Batch_acc: 0.549970. Batch_loss: 1.234282 
Batch: 1504. Acc: 0.564002. Loss: 1.199264. Batch_acc: 0.592825. Batch_loss: 1.133488 
Batch: 1505. Acc: 0.564003. Loss: 1.199268.

KeyboardInterrupt: 

### Los test se realizaron en los cuadernos pruebasPrimerModelo y pruebasSegundoModelo

Esto es solo una muestr ade lo que podemos ejecutar

In [ ]:
model_process.predict_single("Solve 5*w + 3 = -2 for w.", model, device, n_best=1)


In [ ]:
model_process.predict_single("Solve 212 = 56*z - 12 for z.", model, device, n_best=1)


In [ ]:
model_process.predict_single("Solve 2514*m = 2508*m - 24 for m.", model, device, n_best=1)


